In [ ]:
# pip installs
!pip install -q -U datasets peft bitsandbytes transformers trl accelerate sentencepiece

In [ ]:
# imports
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login
import os, wandb
import re
import math
from tqdm import tqdm
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
import transformers.utils.hub
import transformers.utils.generic
import transformers.tokenization_utils_base

In [ ]:
# set check point tracker
# Path to store the checkpoint tracking information
def create_checkpoint_tracker():
    checkpoint_file = "checkpoint_tracker.py"  # File Python nhỏ sinh ra để chứa hàm get/save step

    with open(checkpoint_file, "w") as f:
        f.write("""
def get_latest_step():
    try:
        with open("latest_step.txt", "r") as f:  # Đọc bước train mới nhất được lưu dưới dạng số nguyên
            return int(f.read().strip())
    except:
        return 0

def save_latest_step(step):
    with open("latest_step.txt", "w") as f:  # Ghi đè bước train hiện tại vào file tracker
        f.write(str(step))
""")

create_checkpoint_tracker()
from checkpoint_tracker import get_latest_step, save_latest_step

In [ ]:
# resume training function
def train_or_resume(
    base_model_name,
    hf_model_name,
    train_dataset,
    lora_config,
    steps_per_session=500,
    max_total_steps=1000,
    batch_size=1,
    grad_accum_steps=16,
    save_steps=100
):
    """
    Train (hoặc tiếp tục train) một mô hình ngôn ngữ dạng causal với LoRA và checkpoint.

    Parameters
    ----------
    base_model_name : str
        Tên model gốc trên Hugging Face (ví dụ "meta-llama/Meta-Llama-3.1-8B") dùng làm backbone.
        Để ở dạng tham số để có thể thay đổi sang model khác mà không phải sửa logic bên trong hàm.
    hf_model_name : str
        Tên repo trên Hugging Face Hub (ví dụ "username/project-name") nơi lưu checkpoint / model
        đã fine-tune. Tách riêng với base_model_name để bạn có thể dùng cùng một base model
        cho nhiều project khác nhau.
    train_dataset : datasets.Dataset
        Dataset đã được tiền xử lý và format sẵn cho bài toán tóm tắt. Việc truyền dataset từ
        bên ngoài giúp hàm không phụ thuộc vào cách bạn load dữ liệu (Kaggle, local, v.v.).
    lora_config : LoraConfig
        Cấu hình LoRA (rank r, alpha, dropout, target_modules...). LoRA cho phép fine-tune ít
        tham số hơn so với full fine-tuning, phù hợp khi tài nguyên GPU hạn chế.
    steps_per_session : int, optional
        Số bước train tối đa cho MỖI lần chạy notebook. Tham số này giúp chia quá trình train dài
        thành nhiều phiên nhỏ để phù hợp giới hạn thời gian của Kaggle / Colab.
    max_total_steps : int, optional
        Tổng số bước train mục tiêu cho toàn bộ project. Kết hợp với checkpoint (latest_step) để
        đảm bảo sau nhiều phiên train bạn vẫn dừng đúng mốc mong muốn.
    batch_size : int, optional
        Kích thước batch trên mỗi thiết bị (per-device train batch size). Được đưa ra ngoài để
        dễ cân chỉnh theo dung lượng VRAM của GPU.
    grad_accum_steps : int, optional
        Số bước tích luỹ gradient trước khi cập nhật trọng số. Cho phép mô phỏng batch lớn hơn
        mà không cần tăng batch_size thực tế (tiết kiệm bộ nhớ, nhưng vẫn giữ chất lượng gradient).
    save_steps : int, optional
        Khoảng cách (tính theo số bước train) giữa các lần lưu checkpoint. Đủ nhỏ để có thể
        resume khi notebook bị ngắt, nhưng không quá nhỏ để tránh lưu quá nhiều lần.

    Returns
    -------
    int
        Bước train cuối cùng sau khi phiên train hiện tại kết thúc (dùng để hiển thị tiến độ
        và làm đầu vào cho lần train kế tiếp).
    """
    # Get the latest step we've trained to
    latest_step = get_latest_step()

    # Check if we've already reached the max steps
    if latest_step >= max_total_steps:
        print(f"Training already completed! Reached {latest_step}/{max_total_steps} steps")
        return

    # Calculate how many steps to train in this session
    steps_this_session = min(steps_per_session, max_total_steps - latest_step)
    print(f"Training for {steps_this_session} steps (total progress: {latest_step}/{max_total_steps})")

    # Set up tokenizer
    tokenizer = AutoTokenizer.from_pretrained(
    base_model_name,
    trust_remote_code=True,
    revision="main",
    local_files_only=False
    )
    tokenizer.chat_template = None
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    # Configure quantization
    quant_config = transformers.BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4"
    )

    # Check if we need to resume training
    try:
        if latest_step > 0:
            print(f"Resuming from checkpoint at step {latest_step}")
            # Load from Hugging Face
            base_model = AutoModelForCausalLM.from_pretrained(
                hf_model_name,
                quantization_config=quant_config,
                device_map="auto",
                trust_remote_code=True,
                revision="main",
                local_files_only=False
            )
        else:
            print("Starting training from base model")
            # Start fresh
            base_model = AutoModelForCausalLM.from_pretrained(
                base_model_name,
                quantization_config=quant_config,
                device_map="auto",
                trust_remote_code=True,
                revision="main",
                local_files_only=False
            )
    except Exception as e:
        print(f"Error loading model, starting fresh: {e}")
        base_model = AutoModelForCausalLM.from_pretrained(
            base_model_name,
            quantization_config=quant_config,
            device_map="auto",
        )

    # Configure training parameters
    train_params = SFTConfig(
        output_dir=f"./checkpoints",
        num_train_epochs=1,
        max_steps=steps_this_session,
        per_device_train_batch_size=batch_size,
        gradient_accumulation_steps=grad_accum_steps,
        optim="paged_adamw_32bit",
        save_steps=save_steps,
        logging_steps=20,
        learning_rate=2e-4,
        weight_decay=0.001,
        fp16=False,
        bf16=True,
        max_grad_norm=0.3,
        warmup_ratio=0.03,
        group_by_length=True,
        lr_scheduler_type="cosine",
        push_to_hub=True,
        hub_model_id=hf_model_name,
        hub_private_repo=True
    )

    # Create trainer
    trainer = SFTTrainer(
        model=base_model,
        train_dataset=train_dataset,
        peft_config=lora_config,
        args=train_params,
    )

    # Train the model
    trainer.train()

    # Push to Hugging Face Hub
    trainer.model.push_to_hub(hf_model_name, private=True)

    # Update and save the latest step count
    save_latest_step(latest_step + steps_this_session)

    print(f"Completed training session ({latest_step + steps_this_session}/{max_total_steps} steps)")
    print(f"Model saved to HuggingFace: {hf_model_name}")

    return latest_step + steps_this_session

In [ ]:

# Model and repository names
BASE_MODEL = "meta-llama/Meta-Llama-3.1-8B"  # Model nền 8B được dùng làm backbone để fine-tune cho nhiệm vụ tóm tắt
HF_USER = "calmm-m"  # Tên tài khoản Hugging Face – dùng để ghép thành tên repo đẩy model lên Hub
PROJECT_NAME = "news-summarization"  # Tên project, giúp phân biệt nhiều mô hình khác nhau trên cùng tài khoản
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_NAME}"  # Tên đầy đủ của repo mô hình trên Hugging Face Hub

user_secrets = UserSecretsClient()  # Dùng Kaggle secrets để lấy token một cách an toàn, không hard-code vào notebook
# Login to Hugging Face
hf_token = user_secrets.get_secret("HF_TOKEN")  # Token truy cập Hugging Face, lưu trong Kaggle secrets
login(token=hf_token, add_to_git_credential=True)

# Log in to Weights & Biases
os.environ["WANDB_API_KEY"] = user_secrets.get_secret("WANDB_API_KEY")  # API key của Weights & Biases để log quá trình train
wandb.login()

# Configure Weights & Biases to record against our project
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint"
os.environ["WANDB_WATCH"] = "gradients"

# Load your dataset
from datasets import load_dataset
dataset = load_dataset(f"{HF_USER}/summarization")
# Check the dataset keys to find the correct train dataset key
print(dataset.keys())
train_data = dataset['train'].shuffle(seed=123).select(range(min(5000, len(dataset['train']))))

# LoRA configuration
lora_parameters = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",  # Khai báo đây là bài toán mô hình ngôn ngữ dạng causal (giống chat model)
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Chỉ chèn LoRA vào các projection của attention để tiết kiệm tham số
)

def formatting_func(example):
    # Trích xuất dữ liệu, lưu ý: `completion` là bản gốc, `prompt` là tóm tắt
    # Chúng ta hoán đổi chúng để `prompt` (yêu cầu) là văn bản gốc
    # và `completion` (đáp án) là tóm tắt
    original_text = example['completion'][0]
    summary_text = example['prompt'][0]

    # Định dạng dữ liệu thành một chuỗi duy nhất theo cấu trúc Instruction-based
    # Dấu '###' giúp mô hình phân biệt giữa yêu cầu và câu trả lời
    instruction_text = f"Tóm tắt đoạn văn bản sau:\n\n{original_text}\n\n### Tóm tắt:\n{summary_text}"
    return {"text": instruction_text}

In [ ]:
# Train or resume
current_step = train_or_resume(
    base_model_name=BASE_MODEL,        # Model gốc trên Hugging Face dùng làm backbone để fine-tune
    hf_model_name=HUB_MODEL_NAME,      # Tên repo trên Hugging Face Hub để lưu mô hình đã fine-tune
    train_dataset=train_data,          # Dataset tóm tắt đã được tiền xử lý và format sẵn
    lora_config=lora_parameters,       # Cấu hình LoRA (rank, alpha, dropout, target_modules,...)
    steps_per_session=240,             # Số bước train thực hiện trong MỖI lần chạy notebook Kaggle
    max_total_steps=240,               # Tổng số bước train mục tiêu cho toàn bộ project
    batch_size=1,                      # Batch size nhỏ để vừa với VRAM GPU (kết hợp gradient accumulation)
    grad_accum_steps=8,                # Tích luỹ gradient 8 lần để mô phỏng batch lớn hơn mà không tốn thêm VRAM
    save_steps=60                      # Lưu checkpoint sau mỗi 60 bước để có thể resume nếu session bị ngắt
)

print(f"Current training progress: {current_step}/240 steps")